# Search Engine Results

In [98]:
import os
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.snowball import DanishStemmer
from textblob import TextBlob
import lemmy
from sklearn.metrics import pairwise_kernels
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sentence_transformers import SentenceTransformer, util
import tensorflow as tf
import torch

embedder = SentenceTransformer('distilbert-multilingual-nli-stsb-quora-ranking')

In [99]:
dataset_name = '2020-02-26_2020-03-26_nodups_9k'

In [100]:
base = os.path.abspath('../')

def outname(name):
    return os.path.basename(dataset_name).split('.')[0] + name
    
outname_tfidf = outname('_distances_tfidf.csv')
outname_bert = outname('_encodings_bert.pt')
outname_df = outname('_preprocessed_df.csv')

print('Loading Processed Dataframe')
df = pd.read_csv(f'{base}/data/processed/{outname_df}', index_col=0)

print('Loading TFIDF distances')
%time tfidf = pd.read_csv( f'{base}/data/processed/{outname_tfidf}', index_col=0)

print('Loading BERT distances')
## It is not in Tensor format, but rather numpy array
# bert = pd.read_csv(f'{base}/data/processed/{outname_bert}', index_col=0)
%time bert = torch.load(f'{base}/data/processed/{outname_bert}')

print('\n All datasets loaded.')

Loading Processed Dataframe
Loading TFIDF distances
CPU times: user 1min, sys: 3.53 s, total: 1min 3s
Wall time: 1min 5s
Loading BERT distances
CPU times: user 1.11 ms, sys: 32.4 ms, total: 33.5 ms
Wall time: 49.1 ms

 All datasets loaded.


In [101]:
tfidf = tfidf.reset_index()

In [102]:
## notice the corpus and title_processed
df.head(5)
df.shape

(10140, 8)

In [103]:
def preprocess_text(text):
    # text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = str(text).lower().strip()

    # caveat: this might conflict with the english text
    da_stop_words = stopwords.words('danish')
    stemmer = DanishStemmer()
    lemmatizer = lemmy.load("da")

    # remove plurals
    textblob = TextBlob(text)
    singles = [stemmer.stem(word) for word in textblob.words]

    # remove danish stopwords
    no_stop_words = [word for word in singles if word not in da_stop_words]

    # join text so it can be lemmatized
    joined_text = " ".join(no_stop_words)

    # lemmatization
    final_text = lemmatizer.lemmatize("", joined_text)

    return final_text[0]

## SEARCH QUERY
---

In [135]:
search_query = 'Børne' 
# search query = 'pædagog skov' # Bert is best, but not great overall
# search_query = 'kok' # slighly better than Jobindex
# search_query = 'læge neuropædiatri' # BERT exceeds
k = 10

## Direct custom search results

In [139]:
%%time

def find_direct_results(search_query):
    matching_entries = [df['title_processed'].index[df['title_processed'].str.contains(word, case=False)]
                        .values for word in search_query.split()]
    return list(set(matching_entries[0]).intersection(*matching_entries))

def print_direct_search_results(direct_results):
    agg_direct_results_indexes = []
    print('DIRECT RESULTS:', len(direct_results), '\n')
    for index, result in enumerate(direct_results[:k]):
        print(df['title'][result])
        agg_direct_results_indexes.append(result)
#     return agg_direct_results_indexes

direct_results = find_direct_results(preprocess_text(search_query))
print_direct_search_results(direct_results)

DIRECT RESULTS: 39 

Koordinatorer til Red Barnets Familieoplevelsesklub i Aarhus
Koordinatorer til Red Barnets Familieoplevelsesklub i Aarhus
Pædagog til vores dejlige vuggestue - HOKUS! POKUS! BARNET I FOKUS!
Pædagog til vuggestue i Barndommens Land (barselsvikariat)
CSV søger ny skolesynskonsulent til Københavns blinde og svagtseende skolebørn
Bliv frivillig i Red Barnet Odenses genbrugsbutik
Bliv frivillig i Red Barnet Odenses genbrugsbutik
Bliv frivillig i Red Barnet Odenses genbrugsbutik
pædagog som brænder for vuggestuebørn
Har du hvad der skal til for at blive medleder i Red Barnet Odense?
CPU times: user 25.1 ms, sys: 3.32 ms, total: 28.5 ms
Wall time: 26.8 ms


## TFIDF: Baseline model

In [137]:
%%time

def recommendations(data, results):
    sorted_results = []
    recommendation_indexes = []
    for result in results:
        sorted_distances = data[str(result)].sort_values().iteritems()
        
        for item_index, item_value in enumerate(sorted_distances):
            index, distance = item_value
            sorted_results.append({
                'item_index': index,
                'distance': distance,
                'title': df['title'][index]
            })
            
    agg_sorted_indexes = [x['item_index'] for x in sorted_results]
    ## removing direct_results for all results
    recommendation_indexes = [x for x in agg_sorted_indexes if x not in results]
        
#     return recommendation_indexes
    return agg_sorted_indexes

def display_results(data, name, direct_results):
    print (f'{name}\n')
    for index, recommendation in enumerate(recommendations(data=data, results=direct_results)[:k]):
        print(df['title'][recommendation])

def get_tfidf_recommendations(query):
    if len(direct_results) == 0:
        return None
    display_results(data=tfidf, name='TFIDF', direct_results=direct_results)

get_tfidf_recommendations(query=search_query)

TFIDF

Koordinatorer til Red Barnets Familieoplevelsesklub i Aarhus
Koordinatorer til Red Barnets Familieoplevelsesklub i Aarhus
Vil du skabe gode oplevelser for børn i vores familieoplevelsesklub på Nørrebro?
Bliv koordinator i klub med fokus på friluftsliv og oplevelser i naturen
FAMILIEOPLEVELSESKLUB - Giv sårbare familier i Odense oplevelser for livet
FAMILIEOPLEVELSESKLUB - Giv sårbare familier i Odense oplevelser for livet
FAMILIEOPLEVELSESKLUB - Giv sårbare familier i Odense oplevelser for livet
Hjælp et udsat barn i Århus ind i et godt og aktivt fritidsliv
Hjælp et udsat barn i Århus ind i et godt og aktivt fritidsliv
Red Barnet Aarhus søger frivillige til vores familielejr i uge 31
CPU times: user 8.39 s, sys: 150 ms, total: 8.54 s
Wall time: 8.86 s


# BERT

In [138]:
%%time

def bert_results(bert_embeddings, query):
    embedded_query = embedder.encode(query, convert_to_tensor=True)
    sims = util.pytorch_cos_sim(embedded_query, bert)[0]

    results = sorted(range(len(sims)), key=lambda x: sims[x], reverse=True)[:k]
    ## removing direct_results for all results
    filtered_results = [x for x in results if x not in direct_results]
    return results


print (f'BERT\n')
for recommendation in bert_results(bert_embeddings=bert, query=search_query):
    print(df['title'][recommendation], df['location'][recommendation]) 

BERT

Børneguitar SoundStoreXL.com A/S
Børnehaveleder Gilleleje
Ungdomsleder Frederiksberg C
Pædagog til 11-årig pige Frederikssund
Centerleder til børne- og ungeområdet Haslev
Hjælper til dreng på 13 år København S
Børneinstrument pakker SoundStoreXL.com A/S
Hjælper til højtbegavet 10-årig dreng Kgs. Lyngby
Pædagog til børnehave Viborg
Pædagog til børnehave Viborg
CPU times: user 665 ms, sys: 20.4 ms, total: 685 ms
Wall time: 710 ms


In [126]:
dataset_name = 'jobindex_2019_03_26-2020_03_26'

outname_df_big = outname('_preprocessed_df.csv')

print('Loading Processed Dataframe...')
df_big = pd.read_csv(f'{base}/data/processed/{outname_df_big}', index_col=0)
print('\nDone.')
print(df_big.shape)

Loading Processed Dataframe...

Done.
(273585, 8)


In [130]:
%%time

outname_bert_big = outname('_encodings_bert.pt')
bert_big = torch.load(f'{base}/data/processed/{outname_bert_big}')
bert_big.shape

CPU times: user 1.58 ms, sys: 765 ms, total: 767 ms
Wall time: 790 ms


torch.Size([273585, 768])

In [168]:
%%time

embedded_query= embedder.encode('advokat til ejendomme i Copenhagen', convert_to_tensor=True)
sims_big = util.pytorch_cos_sim(embedded_query, bert_big)[0]
results_bert = sorted(range(len(sims_big)), key=lambda x: sims_big[x], reverse=True)[:k]


for recommendation in results_bert:
    print(df_big['title'][recommendation]) 

Jurist med speciale i fast ejendom til Københavns Ejendomme og Indkøb
Jurist med kendskab til erhvervslejeret og fast ejendom til Udlejning i Københavns Ejendomme & Indkøb
Jurist med kendskab til erhvervslejeret og fast ejendom til Udlejning hos Københavns Ejendomme & Indkøb
Advokatfuldmægtige til København
Erfaren ejendomsadministrator søges til advokatvirksomhed i København
Advokat/jurist med kendskab til offentlig ret og fast ejendom generelt til afdelingen Udlejning i Københavns Kommune
Jurist med kompetencer inden for ejendomsjura til Københavns Ejendomme og Indkøb
Jurist med kompetencer inden for ejendomsjura til Københavns Ejendomme og Indkøb
Jurist med kompetencer inden for ejendomsjura til Københavns Ejendomme og Indkøb
Sagsbehandlere inden for ejendomsskat til Københavns Kommune
CPU times: user 25 s, sys: 913 ms, total: 25.9 s
Wall time: 33.9 s
